 **Ön işleme şablonundan bana lazım olan verileri kopyalayıp buraya yapıştırıyorum.**

In [20]:
#1.kutuphaneler
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#2.veri onisleme
#2.1.veri yukleme
veriler = pd.read_csv('veriler/tenis.csv')
veriler

,outlook,temperature,humidity,windy,play
0,sunny,85,85,False,no
1,sunny,80,90,True,no
2,overcast,83,86,False,yes
3,rainy,70,96,False,yes
4,rainy,68,80,False,yes
5,rainy,65,70,True,no
6,overcast,64,65,True,yes
7,sunny,72,95,False,no
8,sunny,69,70,False,yes
9,rainy,75,80,False,yes


In [21]:
#encoder: Kategorik -> Numeric

# Şimdi burada tek tek numeric'e çevirmek yerine toplu bir şekilde çevirme yapacağız.

from sklearn import preprocessing as pr

veriler2 = veriler.apply(pr.LabelEncoder().fit_transform)
veriler2

,outlook,temperature,humidity,windy,play
0,2,11,4,0,0
1,2,8,6,1,0
2,0,10,5,0,1
3,1,4,9,0,1
4,1,2,3,0,1
5,1,1,1,1,0
6,0,0,0,1,1
7,2,6,8,0,0
8,2,3,1,0,1
9,1,7,3,0,1


Görüldüğü gibi kategorik olan verileri numeriğe çevirdik fakat kullandığımız numeric değerleride dönüştürme yapmış ve biz bu durumu istemiyoruz ve ilk kolonu onehotencoding'e çevirmemiz gerekiyor.

In [22]:
veriler2 = veriler.apply(pr.LabelEncoder().fit_transform)
c = veriler2.iloc[:,:1]

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categorical_features='all')
c = ohe.fit_transform(c).toarray()
print(c)

[[0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:441: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. The passed value of 'all' is the default and can simply be removed.
  DeprecationWarning)


In [24]:
# Şimdi hepsini tek bir DF'de toplayalım.

hava_durumu = pd.DataFrame(data=c, index=range(14), columns=['overcast','rainy','sunny'])
sonveriler = pd.concat([hava_durumu, veriler.iloc[:,1:3]], axis = 1)
sonveriler = pd.concat([veriler2.iloc[:,-2:], sonveriler], axis = 1)
sonveriler

,windy,play,overcast,rainy,sunny,temperature,humidity
0,0,0,0.0,0.0,1.0,85,85
1,1,0,0.0,0.0,1.0,80,90
2,0,1,1.0,0.0,0.0,83,86
3,0,1,0.0,1.0,0.0,70,96
4,0,1,0.0,1.0,0.0,68,80
5,1,0,0.0,1.0,0.0,65,70
6,1,1,1.0,0.0,0.0,64,65
7,0,0,0.0,0.0,1.0,72,95
8,0,1,0.0,0.0,1.0,69,70
9,0,1,0.0,1.0,0.0,75,80


## "humidity"(Nem)i tahmin etme işlemleri

In [25]:
#verilerin egitim ve test icin bolunmesi

from sklearn.model_selection import train_test_split

x_train, x_test,y_train,y_test = train_test_split(sonveriler.iloc[:,:-1],sonveriler.iloc[:,-1:],test_size=0.33, random_state=0)

In [26]:
# Burada yaptığım işlem ile tüm tabloları yan yana görüntüleyebiliriz.

from IPython.core.display import display, HTML

def display_side_by_side(dfs:list, captions:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))

display_side_by_side([x_train,x_test,y_train,y_test],['x_train','x_test','y_train','y_test'])

,windy,play,overcast,rainy,sunny,temperature
13,1,0,0,1,0,71
9,0,1,0,1,0,75
1,1,0,0,0,1,80
7,0,0,0,0,1,72
10,1,1,0,0,1,75
3,0,1,0,1,0,70
0,0,0,0,0,1,85
5,1,0,0,1,0,65
12,0,1,1,0,0,81
,windy,play,overcast,rainy,sunny,temperature


## Çoklu Değişken Linear Model Oluşturma

In [27]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(x_train,y_train)

y_pred = regressor.predict(x_test)
print(y_pred)
y_test

[[84.45365573]
 [63.93839954]
 [85.76050662]
 [64.21013241]
 [75.06793322]]


,humidity
8,70
6,65
4,80
11,90
2,86


##  Geri Eleme (Backward Elimination) Yöntemini Kullanma

In [28]:
import statsmodels.api as sm 

X = np.append(arr = np.ones((14,1)).astype(int), values=sonveriler.iloc[:,:-1], axis=1) # arr içine yazdığım komut bana 22 satır 1 sütunluk matris oluşturuyor, values ile nereye ekleyeceğimi söylüyorum, axis ileyse yan tarafına eklemesini söylüyorum.
X

array([[ 1.,  0.,  0.,  0.,  0.,  1., 85.],
       [ 1.,  1.,  0.,  0.,  0.,  1., 80.],
       [ 1.,  0.,  1.,  1.,  0.,  0., 83.],
       [ 1.,  0.,  1.,  0.,  1.,  0., 70.],
       [ 1.,  0.,  1.,  0.,  1.,  0., 68.],
       [ 1.,  1.,  0.,  0.,  1.,  0., 65.],
       [ 1.,  1.,  1.,  1.,  0.,  0., 64.],
       [ 1.,  0.,  0.,  0.,  0.,  1., 72.],
       [ 1.,  0.,  1.,  0.,  0.,  1., 69.],
       [ 1.,  0.,  1.,  0.,  1.,  0., 75.],
       [ 1.,  1.,  1.,  0.,  0.,  1., 75.],
       [ 1.,  1.,  1.,  1.,  0.,  0., 72.],
       [ 1.,  0.,  1.,  1.,  0.,  0., 81.],
       [ 1.,  1.,  0.,  0.,  1.,  0., 71.]])

In [30]:
X_l = sonveriler.iloc[:,[0,1,2,3,4,5]].values # her bir kolonu ifade edecek bir liste oluşturuyorum 
X_l = np.array(X_l,dtype=float)

model = sm.OLS(sonveriler.iloc[:,-1:],X_l).fit() # Bu OLS işlemi bizim istatiksel verilerimizi çıkartmaya yarıyor, burada humidity ile bağımsız değişkenleri içeren dizi(X_l) üzerindeki bağlantıyı kuruyoruz.
model.summary() # istatistiksel özeti çıkarıcağız

C:\Users\Pc\AppData\Roaming\Python\Python37\site-packages\scipy\stats\stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               humidity   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                 -0.148
Method:                 Least Squares   F-statistic:                    0.6653
Date:                Sat, 03 Oct 2020   Prob (F-statistic):              0.661
Time:                        17:51:15   Log-Likelihood:                -49.542
No. Observations:                  14   AIC:                             111.1
Df Residuals:                       8   BIC:                             114.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -4.0286      7.229     -0.557      0.593     -20.698      12.641
x2            -8.2778      8.029     -1.031      0.333     -26.793      10.237
x3            52.3891     50.214      1.043      0.327     -63.404     168.183
x4            55.6337     45.233      1.230      0.254     -48.673     159.940
x5            49.4291     48.337      1.023      0.336     -62.035     160.893
x6             0.4920      0.597      0.825      0.433      -0.884       1.868
==============================================================================
Omnibus:                        0.935   Durbin-Watson:                   2.416
Prob(Omnibus):                  0.627   Jarque-Bera (JB):                0.823
Skew:                           0.389   Prob(JB):                        0.663
Kurtosis:                       2.103   Cond. No.                     2.08e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.08e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [31]:
sonveriler = sonveriler.iloc[:,1:] # Buradan en yüksek p-değerine sahip olan birinci kolonu çıkarıyorum

X_l = sonveriler.iloc[:,[0,1,2,3,4]].values # buradan da 1 kolonu siliyoruz
X_l = np.array(X_l,dtype=float)

model = sm.OLS(sonveriler.iloc[:,-1:],X_l).fit() # Bu OLS işlemi bizim istatiksel verilerimizi çıkartmaya yarıyor, burada humidity ile bağımsız değişkenleri içeren dizi(X_l) üzerindeki bağlantıyı kuruyoruz.
model.summary() # istatistiksel özeti çıkarıcağız

C:\Users\Pc\AppData\Roaming\Python\Python37\site-packages\scipy\stats\stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               humidity   R-squared:                       0.266
Model:                            OLS   Adj. R-squared:                 -0.060
Method:                 Least Squares   F-statistic:                    0.8165
Date:                Sat, 03 Oct 2020   Prob (F-statistic):              0.546
Time:                        17:54:59   Log-Likelihood:                -49.809
No. Observations:                  14   AIC:                             109.6
Df Residuals:                       9   BIC:                             112.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -6.2865      6.909     -0.910      0.387     -21.916       9.343
x2            36.6564     39.905      0.919      0.382     -53.615     126.927
x3            41.9135     36.464      1.149      0.280     -40.573     124.400
x4            35.1064     39.339      0.892      0.395     -53.884     124.097
x5             0.6484      0.506      1.282      0.232      -0.496       1.793
==============================================================================
Omnibus:                        0.887   Durbin-Watson:                   2.360
Prob(Omnibus):                  0.642   Jarque-Bera (JB):                0.810
Skew:                           0.424   Prob(JB):                        0.667
Kurtosis:                       2.181   Cond. No.                     1.73e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.73e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

------------------
* **Tekrardan tahmin oluşturalım**
* **Burda ilk kolonları silmemiz lazım ben bunu DF ile yapacağım**

In [34]:
x_train = x_train.iloc[:,1:] # Birinci kolonu attım
x_test = x_test.iloc[:,1:] # Birinci sütünu attım

# Şimdi tekrardan tahmn edelim

regressor.fit(x_train,y_train)

y_pred = regressor.predict(x_test)
print(y_pred)
y_test

[[81.86890756]
 [69.08571429]
 [83.46722689]
 [71.86890756]
 [75.69579832]]


,humidity
8,70
6,65
4,80
11,90
2,86


Önceki tahmine oranla gerçek veriye dahada yaklaştık. Bu demek oluyor ki, p-değeri ne kadar yüksek sütun varsa o sütunu verimizden atarsak, yaptığımız tahminin doğruluk oranı o kadar artıyor.